# 📖 Chapter 03 — Vector Database Setup

## 🎯 Objectives

In this chapter, we will build a vector database system for semantic search using ChromaDB and HuggingFace embeddings.

**What we'll accomplish:**

- Set up ChromaDB and embedding models

- Load RAG documents from Chapter 2

- Generate embeddings for all attractions

- Store vectors in ChromaDB collection

- Implement semantic search functionality

- Add metadata filtering capabilities

- Validate retrieval quality

- Create interactive query interface

## 📦 Step 01 — Import Libraries

Import necessary libraries for working with ChromaDB and embeddings.

In [7]:
import json
from pathlib import Path

import chromadb
from chromadb.config import Settings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from src.config import PROCESSED_DATA_DIR, CHROMA_DB_DIR
from src.utils.emoji_log import success, info, error, task, done, warn, data

info("All libraries imported successfully!")

💬 All libraries imported successfully!


## 🧠 Step 02 — Understanding Vector Embeddings

Learn about embeddings and why vector databases are essential for semantic search.

In [3]:
task("Understanding embeddings...")

test_texts = [
    "romantic restaurant with great ambiance",
    "cozy dining place for couples",
    "outdoor hiking trail in mountains",
]

info("These are sample texts we'll convert to vectors:")
for i, text in enumerate(test_texts, 1):
    print(f"  {i}. {text}")

print("\n" + "=" * 70)
info("Key Concepts:")
print("  • Embeddings = numerical representations of text")
print("  • Similar meanings → Similar vectors")
print("  • Vector database = search by meaning, not just keywords")
print("=" * 70)
done("Concept understood! Ready to initialize embedding model.")

🚀 Understanding embeddings...
💬 These are sample texts we'll convert to vectors:
  1. romantic restaurant with great ambiance
  2. cozy dining place for couples
  3. outdoor hiking trail in mountains

💬 Key Concepts:
  • Embeddings = numerical representations of text
  • Similar meanings → Similar vectors
  • Vector database = search by meaning, not just keywords
🏁 Concept understood! Ready to initialize embedding model.


## 🔧 Step 03 — Initialize Embedding Model

Set up HuggingFace embedding model (sentence-transformers/all-MiniLM-L6-v2).

In [ ]:
task("Initializing embedding model...")

# HuggingFaceEmbeddings is a wrapper, it's kinda interface containing lost of embedding models
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={
        "normalize_embeddings": True
    },  # Normalizing Vectorization to boost search quality
)

success("Embedding model loaded successfully!")
info("Model: sentence-transformers/all-MiniLM-L6-v2")
info("Device: CPU")
info("Vector dimensions: 384")

🚀 Initializing embedding model...
✅ Embedding model loaded successfully!
💬 Model: sentence-transformers/all-MiniLM-L6-v2
💬 Device: CPU
💬 Vector dimensions: 384


In [ ]:
task("Testing embedding generation...")

test_text = "Space Needle is an iconic landmark in Seattle"

# Generate embedding (384 dims)
test_vector = embeddings.embed_query(test_text)

success("Embedding generated successfully!")
data("Embedding Details:")
print(f"  • Input text: '{test_text}'")
print(f"  • Vector length: {len(test_vector)} dimensions")
print(f"  • Sample values (first 10): {test_vector[:10]}")
print(f"  • Vector type: {type(test_vector)}")
done("Embedding model is ready to use!")

🚀 Testing embedding generation...
✅ Embedding generated successfully!
📊 Embedding Details:
  • Input text: 'Space Needle is an iconic landmark in Seattle'
  • Vector length: 384 dimensions
  • Sample values (first 10): [0.10850576311349869, 0.039808135479688644, 0.007064886391162872, -0.02410932630300522, -0.0634358748793602, 0.04419279843568802, 0.033649396151304245, -0.04011048376560211, 0.008441255427896976, 0.02948104590177536]
  • Vector type: <class 'list'>
🏁 Embedding model is ready to use!


## 💾 Step 04 — Set Up ChromaDB

Initialize ChromaDB client and create a collection for travel attractions.

In [8]:
task("Setting up ChromaDB...")

collection_name = "travel_attraction"

info(f"Database path: {CHROMA_DB_DIR}")
info(f"Collection name: {collection_name}")

🚀 Setting up ChromaDB...
💬 Database path: c:\Users\dinni\OneDrive\桌面\Travel_rag\chroma_db
💬 Collection name: travel_attraction


In [9]:
# Create a chromaDB client, persistent means saving the data permanently.
client = chromadb.PersistentClient(path=CHROMA_DB_DIR)

success("ChromaDB client created!")

✅ ChromaDB client created!


In [10]:
# Create or retrieve collection
# Collection is similar to a table in DB
collection = client.get_or_create_collection(
    name=collection_name,
    metadata={"description": "Worldwide tourism attractions with embeddings"},
)

success(f"Collection '{collection_name}' ready!")

✅ Collection 'travel_attraction' ready!


In [ ]:
data("Collection Information:")
print(f"  • Name: {collection.name}")
# Return the document amounts in the collection
print(f"  • Total documents: {collection.count()}")
print(f"  • Metadata: {collection.metadata}")
print("=" * 70)
done("ChromaDB setup complete!")

📊 Collection Information:
  • Name: travel_attraction
  • Total documents: 0
  • Metadata: {'description': 'Worldwide tourism attractions with embeddings'}
🏁 ChromaDB setup complete!


## 📄 Step 05 — Load RAG Documents

Load the processed documents from Chapter 2 (seattle_attractions_documents.json).

In [13]:
task("Loading RAG documents from Chapter 2...")

doc_file = PROCESSED_DATA_DIR / "Seattle_attractions_documents.json"

with open(doc_file, "r", encoding="utf-8") as f:
    documents_data = json.load(f)

success(f"Loaded {len(documents_data)} documents")
info(f"File: {doc_file.name}")

print("\n" + "=" * 70)
data("Sample Document Structure:")
print(json.dumps(documents_data[0], indent=2, ensure_ascii=False))
print("=" * 70)
done("Documents loaded successfully!")

🚀 Loading RAG documents from Chapter 2...
✅ Loaded 62 documents
💬 File: Seattle_attractions_documents.json

📊 Sample Document Structure:
{
  "place_id": null,
  "name": "Seattle Public Library - Central Library",
  "document": "Name: Seattle Public Library - Central Library\nLocation: Seattle Central Library, 1000 4th Avenue, Seattle, WA 98104, United States of America\nCoordinates: 47.606714200029515, -122.33269832546111\nDescription: The Seattle Central Library is the flagship library of the Seattle Public Library system. The 11-story (185 feet or 56.9 meters high) glass and steel building in the downtown core of Seattle, Washington was opened to the public on May 23, 2004. Rem Koolhaas and Joshua Prince-Ramus of OMA/LMN were the principal architects, and Magnusson Klemencic Associates was the structural engineer with Arup. Arup also provided mechanical, electrical, and plumbing engineering, as well as fire/life safety, security, IT and communications, and audio visual consulting. Ho

## 🏗️ Step 06 — Prepare Document Metadata

Extract and structure metadata for each document (place_id, name, location, etc.).

## 🚀 Step 07 — Generate Embeddings

Generate vector embeddings for all attraction documents using batch processing.

## 💿 Step 08 — Store in ChromaDB

Add documents, embeddings, and metadata to the ChromaDB collection.

## 🔍 Step 09 — Test Basic Semantic Search

Perform test queries to verify semantic search functionality.

## 🎯 Step 10 — Implement Metadata Filtering

Add filtering capabilities (by city, category, coordinates).

## 📊 Step 11 — Analyze Search Results

Examine similarity scores and result quality for various queries.

## ✅ Step 12 — Validate Retrieval Quality

Run test cases to ensure search accuracy and relevance.

## 📈 Step 13 — Collection Statistics

View collection stats (document count, embedding dimensions, storage size).

## 🖥️ Step 14 — Create Query Function

Build a reusable function for interactive querying.

## 💾 Step 15 — Save Collection

Verify that ChromaDB persists data correctly.

## 🧪 Step 16 — Final Testing

Comprehensive testing with diverse queries and edge cases.

## 📋 Step 17 — Summary and Next Steps

Review what we've built and prepare for Chapter 4 (RAG Pipeline with LLM).